In [1]:
import random
import re

In [2]:
with open('25.txt', 'r') as file:
    data = [int(s) for s in file.read().split(',')]


In [3]:
class Halt(Exception):
    pass


class Yield(Exception):
    pass


class VM:
    def __init__(self, memory, inputs, outputs):
        self.ops = {1: self.add, 
                    2: self.mul, 
                    3: self.input_, 
                    4: self.output, 
                    5: self.jnz,
                    6: self.jz,
                    7: self.lt,
                    8: self.eq,
                    9: self.rbo,
                    99: self.hcf}
        self.memory = memory
        self.pointer = 0
        self.base = 0
        self.inputs = inputs
        self.outputs = outputs

    def run(self):
        while self.step():
            pass
        
    def step(self):
        try:
            while True:
                self.ops[self.read(self.pointer) % 100]()
        except Yield:
            return True
        except Halt:
            return False

    def add(self):
        a, b, c = self.decode(3)
        self.write(c, self.read(a) + self.read(b))

    def mul(self):
        a, b, c = self.decode(3)
        self.write(c, self.read(a) * self.read(b))

    def input_(self):
        if not self.inputs:
            raise Yield()
        a, = self.decode(1)
        self.write(a, self.inputs.pop(0))

    def output(self):
        a, = self.decode(1)
        self.outputs.append(self.read(a))

    def jnz(self):
        a, b = self.decode(2)
        if self.read(a) != 0:
            self.pointer = self.read(b)

    def jz(self):
        a, b = self.decode(2)
        if self.read(a) == 0:
            self.pointer = self.read(b)

    def lt(self):
        a, b, c = self.decode(3)
        self.write(c, 1 if self.read(a) < self.read(b) else 0)

    def eq(self):
        a, b, c = self.decode(3)
        self.write(c, 1 if self.read(a) == self.read(b) else 0)
        
    def rbo(self):
        a, = self.decode(1)
        self.base += self.read(a)

    def hcf(self):
        raise Halt()

    def decode(self, count):
        mode = self.read(self.pointer) // 100
        self.pointer += 1

        params = []
        for i in range(count):
            value = self.pointer
            if mode % 10 == 0:
                value = self.read(value)
            elif mode % 10 == 2:
                value = self.base + self.read(value)
            params.append(value)
            self.pointer += 1
            mode //= 10
        return params
    
    def read(self, offset):
        self.extend(offset)
        return self.memory[offset]
    
    def write(self, offset, value):
        self.extend(offset)
        self.memory[offset] = value
    
    def extend(self, offset):
        if offset >= len(self.memory):
            self.memory.extend([0] * (offset - len(self.memory) + 1))


In [30]:
BAD_ITEMS = {'infinite loop', 'giant electromagnet', 'escape pod', 'molten lava', 'photons'}

def nextpos(pos, dir_):
    x, y = pos
    if dir_ == 'north':
        return x, y + 1
    elif dir_ == 'south':
        return x, y - 1
    elif dir_ == 'east':
        return x + 1, y
    elif dir_ == 'west':
        return x - 1, y
    
random.seed(0)

pos = 0, 0
visited = {pos}
inputs = []
outputs = []
vm = VM(data.copy(), inputs, outputs)
while True:
    vm.step()
    msg = ''.join(chr(o) for o in outputs)
    outputs.clear()

    if '== Pressure-Sensitive Floor ==' in msg:
        print(msg)
        break        
    elif '== Security Checkpoint ==' in msg:
        inputs += [ord(c) for c in f'drop whirled peas\n']
        command = 'north'
    else:
        dirs = re.findall(r'Doors here lead:(.*?)\n\n', msg, re.MULTILINE | re.DOTALL)
        items = re.search(r'Items here:(.*?)\n\n', msg, re.MULTILINE | re.DOTALL)
        if items:
            for item in re.findall(r'^- (.*)$', items.group(0), re.MULTILINE):
                if item not in BAD_ITEMS:
                    inputs += [ord(c) for c in f'take {item}\n']
        commands = [r for g in dirs for r in re.findall(r'^- (.*)$', g, re.MULTILINE)]
        commands = [c for c in commands if nextpos(pos, c) not in visited] or commands
        command = random.choice(commands)
        
    pos = nextpos(pos, command)
    visited.add(pos)
    inputs += [ord(c) for c in command + '\n']

inputs += [ord(c) for c in 'inv\n']
vm.step()
msg = ''.join(chr(o) for o in outputs)
print(msg)



You drop the whirled peas.

Command?



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- south

A loud, robotic voice says "Analysis complete! You may proceed." and you enter the cockpit.
Santa notices your small droid, looks puzzled for a moment, realizes what has happened, and radios your ship directly.
"Oh, hello! You should be able to get in by typing 16810049 on the keypad at the main airlock."


